# Bird Migration Routes on NASA Nightlights Imagery

This notebook combines:
1. NASA VIIRS nightlights satellite data (VNP46A3)
2. Bird migration route GPS coordinates
3. Proper latitude/longitude georeferencing

The result is migration routes overlaid on nighttime light pollution maps.

## Step 1: Import Required Libraries

In [3]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize, LogNorm
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## Step 1.5: Determine Which Nightlights Tiles You Need (Helper)

This optional cell helps you figure out which VIIRS tiles cover your migration data.
Run this before Step 2 to see which h/v tile numbers you need to download.

In [8]:
def lat_lon_to_modis_tile(lat, lon):
    """
    Convert lat/lon coordinates to MODIS/VIIRS tile numbers (h, v).
    
    Based on MODIS sinusoidal projection.
    """
    # Earth radius
    R = 6371007.181
    tile_size = 1111950.5197665554
    
    # Convert lat/lon to meters
    lat_rad = lat * np.pi / 180
    lon_rad = lon * np.pi / 180
    
    x_meters = R * lon_rad * np.cos(lat_rad)
    y_meters = R * lat_rad
    
    # Convert to tile coordinates
    h = int(np.floor((x_meters + 18 * tile_size) / tile_size))
    v = int(np.floor((9 * tile_size - y_meters) / tile_size))
    
    return h, v

# Load your migration data first to see what tiles you need
migration_file = '/Users/dazedinthecity/Documents/GitHub/ceu-ds-project-groupB-2026/historic_bird/setophaga-plot-1102.csv'  # UPDATE THIS PATH

try:
    df_check = pd.read_csv(migration_file)
    
    # Get all unique tiles needed
    tiles_needed = set()
    for idx, row in df_check.iterrows():
        if pd.notna(row['GPS_xx']) and pd.notna(row['GPS_yy']):
            h, v = lat_lon_to_modis_tile(row['GPS_yy'], row['GPS_xx'])
            tiles_needed.add((h, v))
    
    tiles_needed = sorted(tiles_needed)
    
    print(f"Your migration data requires {len(tiles_needed)} VIIRS tile(s):\n")
    for h, v in tiles_needed:
        print(f"  h{h:02d}v{v:02d}")
    
    print("\nYou need to download files matching these patterns:")
    for h, v in tiles_needed:
        print(f"*.h{h:02d}v{v:02d}.*.h5",',')
    
    print("\nDownload these tiles from NASA LAADS DAAC:")
    print("https://ladsweb.modaps.eosdis.nasa.gov/search/")
    
except FileNotFoundError:
    print(f"Migration file not found. Please update the path above.")
except Exception as e:
    print(f"Error: {e}")

Your migration data requires 18 VIIRS tile(s):

  h10v05
  h10v06
  h10v07
  h10v08
  h10v09
  h11v05
  h11v06
  h11v07
  h11v08
  h11v09
  h11v10
  h12v03
  h12v04
  h12v05
  h12v10
  h12v11
  h13v03
  h13v04

You need to download files matching these patterns:
*.h10v05.*.h5 ,
*.h10v06.*.h5 ,
*.h10v07.*.h5 ,
*.h10v08.*.h5 ,
*.h10v09.*.h5 ,
*.h11v05.*.h5 ,
*.h11v06.*.h5 ,
*.h11v07.*.h5 ,
*.h11v08.*.h5 ,
*.h11v09.*.h5 ,
*.h11v10.*.h5 ,
*.h12v03.*.h5 ,
*.h12v04.*.h5 ,
*.h12v05.*.h5 ,
*.h12v10.*.h5 ,
*.h12v11.*.h5 ,
*.h13v03.*.h5 ,
*.h13v04.*.h5 ,

Download these tiles from NASA LAADS DAAC:
https://ladsweb.modaps.eosdis.nasa.gov/search/


## Step 2: Load and Process NASA Nightlights Data

Extract nightlights data from multiple HDF5 files and mosaic them together to cover the entire migration trajectory.

In [ ]:
import os
import re
from collections import defaultdict

# MODIFY THIS to point to your nightlights HDF5 files
# Option 1: List specific files
nightlights_files = [
    '/path/to/VNP46A3.A2014182.h19v04.002.2025090185911.h5',
    '/path/to/VNP46A3.A2014182.h20v04.002.2025090185911.h5',
    '/path/to/VNP46A3.A2014182.h19v05.002.2025090185911.h5',
]

# Option 2: Use a directory with pattern matching (uncomment to use)
# nightlights_dir = '/path/to/nightlights/folder'
# nightlights_pattern = 'VNP46A3.*.h5'  # Pattern to match files
# nightlights_files = [os.path.join(nightlights_dir, f) for f in os.listdir(nightlights_dir) 
#                      if re.match(nightlights_pattern.replace('*', '.*'), f)]

def extract_tile_info(filename):
    """
    Extract h and v tile numbers from filename.
    Example: VNP46A3.A2014182.h19v04.002.h5 -> (19, 4)
    """
    match = re.search(r'h(\d+)v(\d+)', os.path.basename(filename))
    if match:
        return int(match.group(1)), int(match.group(2))
    raise ValueError(f"Could not extract tile info from filename: {filename}")

def load_single_tile(filename):
    """
    Load a single nightlights tile with coordinates.
    
    Returns:
        dict with 'data', 'lat', 'lon', 'h_tile', 'v_tile'
    """
    h_tile, v_tile = extract_tile_info(filename)
    
    with h5py.File(filename, 'r') as f:
        # Get the data structure
        data_key = list(f['HDFEOS/GRIDS'].keys())[0]  # Usually 'VNP_Grid_DNB'
        data = f[f'HDFEOS/GRIDS/{data_key}/Data Fields']
        
        # Get nightlights data
        if 'NearNadir_Composite_Snow_Free' in data:
            ntl_data = data['NearNadir_Composite_Snow_Free'][:]
        else:
            # Use first available dataset
            dataset_name = list(data.keys())[0]
            ntl_data = data[dataset_name][:]
        
        nrows, ncols = ntl_data.shape
        
        # MODIS/VIIRS sinusoidal grid parameters
        tile_size = 1111950.5197665554  # meters per tile
        pixel_size = tile_size / nrows
        R = 6371007.181  # Earth radius in meters
        
        # Create pixel coordinate arrays
        x = np.arange(ncols)
        y = np.arange(nrows)
        xx, yy = np.meshgrid(x, y)
        
        # Convert to sinusoidal projection meters
        x_meters = (h_tile * tile_size) + (xx * pixel_size) - (18 * tile_size)
        y_meters = (9 * tile_size) - (v_tile * tile_size) - (yy * pixel_size)
        
        # Convert to lat/lon (WGS84)
        lat = y_meters / R * (180 / np.pi)
        lon = x_meters / (R * np.cos(lat * np.pi / 180)) * (180 / np.pi)
        
        print(f"  h{h_tile:02d}v{v_tile:02d}: Lat [{lat.min():.2f}, {lat.max():.2f}], "
              f"Lon [{lon.min():.2f}, {lon.max():.2f}], "
              f"Valid pixels: {np.sum(ntl_data > 0):,}")
        
        return {
            'data': ntl_data,
            'lat': lat,
            'lon': lon,
            'h_tile': h_tile,
            'v_tile': v_tile
        }

def mosaic_tiles(tiles):
    """
    Mosaic multiple tiles into a single combined array.
    
    Returns:
        ntl_data: Combined nighttime lights array
        lat: Combined latitude array
        lon: Combined longitude array
    """
    if len(tiles) == 1:
        # Single tile, no mosaicking needed
        return tiles[0]['data'], tiles[0]['lat'], tiles[0]['lon']
    
    # Organize tiles by position
    tile_dict = {}
    for tile in tiles:
        tile_dict[(tile['h_tile'], tile['v_tile'])] = tile
    
    # Find bounds
    h_tiles = [t['h_tile'] for t in tiles]
    v_tiles = [t['v_tile'] for t in tiles]
    h_min, h_max = min(h_tiles), max(h_tiles)
    v_min, v_max = min(v_tiles), max(v_tiles)
    
    print(f"\nMosaicking {len(tiles)} tiles: h[{h_min}-{h_max}] x v[{v_min}-{v_max}]")
    
    # Get tile dimensions from first tile
    first_tile = tiles[0]
    tile_height, tile_width = first_tile['data'].shape
    
    # Create output arrays
    n_rows = (v_max - v_min + 1) * tile_height
    n_cols = (h_max - h_min + 1) * tile_width
    
    mosaic_data = np.zeros((n_rows, n_cols), dtype=first_tile['data'].dtype)
    mosaic_lat = np.zeros((n_rows, n_cols), dtype=first_tile['lat'].dtype)
    mosaic_lon = np.zeros((n_rows, n_cols), dtype=first_tile['lon'].dtype)
    
    # Fill in each tile
    for (h, v), tile in tile_dict.items():
        # Calculate position in mosaic
        row_start = (v - v_min) * tile_height
        row_end = row_start + tile_height
        col_start = (h - h_min) * tile_width
        col_end = col_start + tile_width
        
        # Place tile data
        mosaic_data[row_start:row_end, col_start:col_end] = tile['data']
        mosaic_lat[row_start:row_end, col_start:col_end] = tile['lat']
        mosaic_lon[row_start:row_end, col_start:col_end] = tile['lon']
    
    print(f"Mosaic shape: {mosaic_data.shape}")
    print(f"Combined coverage: Lat [{mosaic_lat.min():.2f}, {mosaic_lat.max():.2f}], "
          f"Lon [{mosaic_lon.min():.2f}, {mosaic_lon.max():.2f}]")
    print(f"Total valid pixels: {np.sum(mosaic_data > 0):,}")
    
    return mosaic_data, mosaic_lat, mosaic_lon

# Load all tiles
print(f"Loading {len(nightlights_files)} nightlights tile(s)...\n")
tiles = []
for i, filename in enumerate(nightlights_files, 1):
    print(f"[{i}/{len(nightlights_files)}] Loading {os.path.basename(filename)}")
    try:
        tile = load_single_tile(filename)
        tiles.append(tile)
    except Exception as e:
        print(f"  ⚠️ Error loading {filename}: {e}")

if len(tiles) == 0:
    raise ValueError("No tiles loaded successfully. Check your file paths.")

# Mosaic tiles together
ntl_data, lat_grid, lon_grid = mosaic_tiles(tiles)

print("\n" + "="*70)
print("✓ Nightlights data loaded and mosaicked successfully!")

## Step 2.5: Visualize Tile Coverage (Optional)

Quick check to see the coverage area of your nightlights tiles.

In [ ]:
# Quick visualization of nightlights coverage
fig, ax = plt.subplots(figsize=(12, 8))

# Plot nightlights with log scale
ntl_display = ntl_data.copy()
ntl_display[ntl_display <= 0] = 0.01

im = ax.pcolormesh(lon_grid, lat_grid, ntl_display, 
                   cmap='gray', 
                   norm=LogNorm(vmin=0.1, vmax=ntl_display.max()),
                   shading='auto',
                   alpha=0.9)

plt.colorbar(im, ax=ax, label='Nighttime Radiance (nW/cm²/sr)')

ax.set_xlabel('Longitude (°E)', fontsize=12, fontweight='bold')
ax.set_ylabel('Latitude (°N)', fontsize=12, fontweight='bold')
ax.set_title(f'Nightlights Coverage Area ({len(tiles)} tile(s))', 
             fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5, color='cyan')

plt.tight_layout()
plt.show()

print(f"\nCoverage Summary:")
print(f"  Latitude:  {lat_grid.min():.2f}° to {lat_grid.max():.2f}°")
print(f"  Longitude: {lon_grid.min():.2f}° to {lon_grid.max():.2f}°")
print(f"  Data shape: {ntl_data.shape}")
print(f"  Tiles loaded: {len(tiles)}")

## Step 3: Load Bird Migration Data

Load the bird migration dataset with GPS coordinates.

In [ ]:
# MODIFY THIS PATH to point to your migration CSV file
migration_file = '/path/to/your/migration_data.csv'

# Load migration data
df = pd.read_csv(migration_file)

print(f"Migration dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

## Step 4: Filter Migration Routes Within Nightlights Coverage

Select only migration points that fall within the geographic bounds of the nightlights data.

In [ ]:
# Get nightlights bounds
lat_min, lat_max = lat_grid.min(), lat_grid.max()
lon_min, lon_max = lon_grid.min(), lon_grid.max()

print(f"Nightlights coverage:")
print(f"  Latitude: {lat_min:.2f}° to {lat_max:.2f}°")
print(f"  Longitude: {lon_min:.2f}° to {lon_max:.2f}°")

# Filter migration data to this region
df_filtered = df[
    (df['GPS_yy'] >= lat_min) & (df['GPS_yy'] <= lat_max) &
    (df['GPS_xx'] >= lon_min) & (df['GPS_xx'] <= lon_max)
].copy()

print(f"\nMigration points in coverage area: {len(df_filtered)} / {len(df)}")

if len(df_filtered) > 0:
    print(f"\nGenera in coverage:")
    for genus in df_filtered['Bird genera'].value_counts().head(10).items():
        print(f"  {genus[0]}: {genus[1]} points")
else:
    print("\n⚠️ Warning: No migration points found in this tile's coverage.")
    print("You may need to:")
    print("  1. Use a different nightlights tile (different h/v values)")
    print("  2. Or use a different migration dataset")

## Step 5: Visualize Single Genus on Nightlights

Create a visualization showing migration routes overlaid on the nightlights imagery.

In [ ]:
def plot_migration_on_nightlights(ntl_data, lat_grid, lon_grid, migration_df, genus_name=None):
    """
    Plot migration routes overlaid on nightlights imagery.
    
    Parameters:
        ntl_data: Nighttime lights array
        lat_grid, lon_grid: 2D coordinate grids
        migration_df: DataFrame with migration data
        genus_name: Specific genus to plot (None = plot all)
    """
    fig, ax = plt.subplots(figsize=(16, 12))
    
    # Filter data
    if genus_name:
        plot_df = migration_df[migration_df['Bird genera'] == genus_name].copy()
        title_suffix = f"{genus_name} Migration Routes"
    else:
        plot_df = migration_df.copy()
        title_suffix = "All Migration Routes"
    
    plot_df = plot_df.dropna(subset=['GPS_xx', 'GPS_yy', 'Migration start year'])
    
    if len(plot_df) == 0:
        print(f"No valid data for {genus_name if genus_name else 'any genus'}")
        return
    
    # Plot nightlights as background (log scale for better visibility)
    # Replace invalid values with a small positive number
    ntl_display = ntl_data.copy()
    ntl_display[ntl_display <= 0] = 0.01
    
    im = ax.pcolormesh(lon_grid, lat_grid, ntl_display, 
                       cmap='gray', 
                       norm=LogNorm(vmin=0.1, vmax=ntl_display.max()),
                       shading='auto',
                       alpha=0.8)
    
    # Add colorbar for nightlights
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label('Nighttime Radiance (nW/cm²/sr)', fontsize=12, rotation=270, labelpad=20)
    
    # Plot migration routes by year
    years = sorted(plot_df['Migration start year'].unique())
    colormap = cm.get_cmap('plasma', len(years))
    norm = Normalize(vmin=min(years), vmax=max(years))
    
    legend_entries = {}
    
    for year in years:
        year_data = plot_df[plot_df['Migration start year'] == year].copy()
        color = colormap(norm(year))
        
        # Plot each unique route
        for route_code in year_data['Migratory route codes'].unique():
            route_data = year_data[year_data['Migratory route codes'] == route_code].sort_values('ID')
            
            # Draw connecting lines between points
            if len(route_data) > 1:
                points = route_data[['GPS_xx', 'GPS_yy']].values
                ax.plot(points[:, 0], points[:, 1], 
                       color=color, linewidth=2.5, alpha=0.9, zorder=4)
            
            # Plot points
            ax.scatter(route_data['GPS_xx'], route_data['GPS_yy'], 
                      c=[color], s=60, 
                      alpha=0.95, edgecolors='white', linewidth=1.5, zorder=5)
            
            # Add to legend (one entry per year)
            if str(int(year)) not in legend_entries:
                legend_entries[str(int(year))] = plt.Line2D([0], [0], 
                                                            color=color, 
                                                            linewidth=3, 
                                                            marker='o', 
                                                            markersize=8, 
                                                            alpha=0.9)
    
    # Labels and formatting
    ax.set_xlabel('Longitude (°E)', fontsize=14, fontweight='bold')
    ax.set_ylabel('Latitude (°N)', fontsize=14, fontweight='bold')
    
    species_info = ""
    if genus_name and 'English Name' in plot_df.columns:
        species_name = plot_df['English Name'].unique()[0]
        species_info = f"\n{species_name}"
    
    ax.set_title(f'{title_suffix} on Nighttime Light Pollution{species_info}\n{len(plot_df)} migration points', 
                 fontsize=18, fontweight='bold', pad=20)
    
    # Add legend for years
    if legend_entries:
        ax.legend(legend_entries.values(), legend_entries.keys(), 
                 title='Migration Year', 
                 loc='upper left', bbox_to_anchor=(1.12, 1), 
                 fontsize=11, title_fontsize=12, framealpha=0.98)
    
    # Set reasonable bounds
    margin = 1  # degrees
    ax.set_xlim(plot_df['GPS_xx'].min() - margin, plot_df['GPS_xx'].max() + margin)
    ax.set_ylim(plot_df['GPS_yy'].min() - margin, plot_df['GPS_yy'].max() + margin)
    
    # Grid for reference
    ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5, color='cyan')
    
    plt.tight_layout()
    
    return fig, ax

# Example: Plot a specific genus if available
if len(df_filtered) > 0:
    # Get the genus with most points
    top_genus = df_filtered['Bird genera'].value_counts().index[0]
    print(f"Plotting {top_genus}...")
    
    fig, ax = plot_migration_on_nightlights(ntl_data, lat_grid, lon_grid, 
                                           df_filtered, genus_name=top_genus)
    
    # Save the figure
    output_filename = f'{top_genus.lower()}_on_nightlights.png'
    plt.savefig(output_filename, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"✓ Saved: {output_filename}")
    plt.show()
else:
    print("No migration data in this region to plot.")

## Step 6: Batch Process All Genera

Create individual maps for each genus in the filtered dataset.

In [ ]:
if len(df_filtered) > 0:
    # Get all unique genera with sufficient data
    genera_counts = df_filtered['Bird genera'].value_counts()
    genera_to_plot = genera_counts[genera_counts >= 3].index.tolist()
    
    print(f"Creating maps for {len(genera_to_plot)} genera...\n")
    
    for i, genus in enumerate(genera_to_plot, 1):
        print(f"[{i}/{len(genera_to_plot)}] Processing {genus}...")
        
        fig, ax = plot_migration_on_nightlights(ntl_data, lat_grid, lon_grid, 
                                               df_filtered, genus_name=genus)
        
        if fig is not None:
            output_filename = f'{genus.lower()}_nightlights_map.png'
            plt.savefig(output_filename, dpi=300, bbox_inches='tight', facecolor='white')
            plt.close(fig)
            print(f"  ✓ Saved: {output_filename}")
    
    print("\n" + "=" * 70)
    print("✓ All maps created successfully!")
else:
    print("No genera to plot in this nightlights tile.")

## Step 7: Create Overview Map (All Routes)

Show all migration routes in the region on a single map.

In [ ]:
if len(df_filtered) > 0:
    print("Creating overview map with all migration routes...")
    
    fig, ax = plot_migration_on_nightlights(ntl_data, lat_grid, lon_grid, 
                                           df_filtered, genus_name=None)
    
    if fig is not None:
        output_filename = 'all_routes_on_nightlights.png'
        plt.savefig(output_filename, dpi=300, bbox_inches='tight', facecolor='white')
        print(f"✓ Saved: {output_filename}")
        plt.show()
else:
    print("No data to create overview map.")

## Step 8: Analysis - Migration Routes vs Light Pollution

Analyze the relationship between migration routes and nighttime light intensity.

In [ ]:
def extract_light_values_at_migration_points(ntl_data, lat_grid, lon_grid, migration_df):
    """
    Extract nightlight values at migration point locations.
    """
    light_values = []
    
    for idx, row in migration_df.iterrows():
        lon_point = row['GPS_xx']
        lat_point = row['GPS_yy']
        
        # Find nearest grid point
        distance = np.sqrt((lon_grid - lon_point)**2 + (lat_grid - lat_point)**2)
        min_idx = np.unravel_index(distance.argmin(), distance.shape)
        
        light_value = ntl_data[min_idx]
        light_values.append(light_value)
    
    return np.array(light_values)

if len(df_filtered) > 0:
    print("Analyzing light pollution at migration points...\n")
    
    # Extract light values
    df_filtered['light_intensity'] = extract_light_values_at_migration_points(
        ntl_data, lat_grid, lon_grid, df_filtered
    )
    
    # Summary statistics
    print("Light Intensity at Migration Points:")
    print(f"  Mean: {df_filtered['light_intensity'].mean():.2f} nW/cm²/sr")
    print(f"  Median: {df_filtered['light_intensity'].median():.2f} nW/cm²/sr")
    print(f"  Std Dev: {df_filtered['light_intensity'].std():.2f} nW/cm²/sr")
    print(f"  Min: {df_filtered['light_intensity'].min():.2f} nW/cm²/sr")
    print(f"  Max: {df_filtered['light_intensity'].max():.2f} nW/cm²/sr")
    
    # Plot histogram
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.hist(df_filtered['light_intensity'][df_filtered['light_intensity'] > 0], 
            bins=50, edgecolor='black', alpha=0.7)
    ax.set_xlabel('Nighttime Light Intensity (nW/cm²/sr)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Number of Migration Points', fontsize=12, fontweight='bold')
    ax.set_title('Distribution of Light Pollution at Migration Waypoints', 
                 fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('light_pollution_histogram.png', dpi=300, bbox_inches='tight')
    print("\n✓ Saved: light_pollution_histogram.png")
    plt.show()
    
    # By genus
    print("\nMean Light Intensity by Genus:")
    genus_light = df_filtered.groupby('Bird genera')['light_intensity'].agg(['mean', 'count'])
    genus_light = genus_light.sort_values('mean', ascending=False)
    print(genus_light.head(10))
else:
    print("No data for light pollution analysis.")

## Summary

This notebook demonstrates:
1. ✅ Loading multiple NASA VIIRS nightlights HDF5 tiles
2. ✅ Mosaicking tiles to cover entire migration trajectories
3. ✅ Computing proper latitude/longitude coordinates for each tile
4. ✅ Overlaying bird migration routes on nightlights imagery
5. ✅ Color-coding routes by year
6. ✅ Analyzing light pollution exposure at migration points

### Multi-Tile Support:
- **Automatic tile detection**: Helper cell identifies which tiles you need
- **Flexible loading**: Specify files individually or load entire directories
- **Seamless mosaicking**: Tiles are automatically stitched together
- **Proper coordinate alignment**: Each tile's lat/lon grid is computed independently

### Next Steps:
- Use multiple nightlights tiles to cover large migration corridors
- Compare different time periods (monthly composites)
- Create temporal animations showing migration through the year
- Statistical analysis of light pollution impact on migration patterns
- Compare different species' exposure to artificial light
- Export results to GeoTIFF for use in GIS software

### Notes:
- The VNP46A3 product provides monthly composites
- Coordinates are computed using the MODIS sinusoidal projection
- Log scaling is used for nightlights to enhance visibility
- Migration routes are connected by year and route code
- Tiles are named with format: VNP46A3.AYYYYDDD.hHHvVV.VVV.YYYYDDDHHMMSS.h5
  - HH = horizontal tile number (0-35)
  - VV = vertical tile number (0-17)